### Jupyter Notebook Overview
What is happening behind the scenes...

![fig2](./graphics/jpynb_BlockDiag.png)

We are using the iJava (Interactive Java Kernel for Jupyter Notebook) in this example.

We also have the option to code in Python - in a separate Notebook, iPython is the other interactive Kernel available in Jupyter Notebooks.

##### Jupyter Notebook - Cells and Execution / Edit shortcuts 

We have a mix of Markdown cells (text description in Markdown) and Code cells (we have chosen iJava Kernel).
<img src="./graphics/JupNb_Cells.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="250"
     height="400"/>


You can use the menu above to change a cell from being a Code cell to a Markdown cell.

**Cntrl-Enter** executes a cell. Try it.

Clicking anywhere in text display (Markdown) cell and hitting **Enter**, puts you in the cell edit mode. Try it.

Code cells can be edited directly. To run an edited code cell, just hit **Cntrl-Enter**.

You can add a new blank cell, cells can be deleted, copied, pasted or moved up or down using the menu icons as shown below:

![fig4](./graphics/JupNb_CellEditing.png)

### Housekeeping - Wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

### Runing _asadm_ via Terminal Tab

Alternatively, you can also open a separate terminal window (TAB) from the Home Page and run _asadm_ interactively in the Terminal.  Please **DO NOT CLICK ON QUIT** on Login Page. It will kill the Jupyter Notebook Server.

![fig5](./graphics/JupNb_Terminal.png)

### Get our Aerospike Java Client talking to the Single Node Aerospike Server 

We will be discussing the data model development interactively. We will code as we progress through the discussion. So let us connect our client application's interactive coding environment with the single node Aerospike server. This will get our real time interaction started.

As we write code in following Jupyter Notebook cells, it will be executed on the Single node Aerospike Server. 

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>9.0.3</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [4]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Initialized the client and connected to the cluster.


# Accessing a record on the Aerospike Server
We build the Key object in Java
We need the namespace and the record digest to find the record on the server.
The record digest is computed by the client library using application provided key (integer, string or byte-array) and the set name. If not in a set, use null for set name.

<img src="./graphics/RecordKey.png"
     alt="Record Digest"
     style="float: left; margin-right: 10px;"
     width="600"
     height="400"/>

In [6]:
Key key = new Key("test", "demo", "key1");
System.out.println("Working with record key:");
System.out.println(key);  //Will show the 20 byte digest

Working with record key:
test:demo:key1:ec91192d4b7f8ce35d5d78d34bca65cbaaaac960


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [7]:
//Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.

WritePolicy wPolicy = new WritePolicy();

Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));

client.put(wPolicy, key, b1, b2);

##### About Jupyter Notebook Cell Execution sequence

If a previous code cell was executed, objects instantiated in it are available in the next cell that you execute. 

You can execute individual cells in any order - this is interactive execution.

Order of execution will determine the net result.  i.e. you can go back and forth between the record insertion cell above, and record read cell below. 

![fig_exec](./graphics/JupNb_Execution.png)

If you edit the insertion values, you must re-execute the write cell to put the new value on the Aerospike Server. Then, if you execute the read cell, you will see the updated value.

**_If you have lost track about where you are in the sequence, go to the top Menu, under Cell, Choose "Run All" to run all cells - top to bottom - in this notebook's sequence._** If you want to clear all objects in the kernel and restart from a fresh state, use **Kernel -> Restart & Run All**.


In [8]:
//Read the record that we just inserted

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Read back the record.
Record values are:
(gen:1),(exp:475970072),(bins:(b1:val1),(b2:2))


## Clean up

In [9]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"